In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My Drive/video_OD

/content/gdrive/My Drive/video_OD


In [3]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from sklearn.model_selection import KFold,StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

In [4]:
train = pd.read_csv('./train/train.csv')
test = pd.read_csv('./test/test.csv')

In [5]:
#first_row = pd.DataFrame({'id':1, 'date':'2022-05-24 08:08:34', 'employee': -1, 'action':-1}, index=[0])
#train = pd.concat([first_row,train.loc[:]]).reset_index(drop=True)

#first_row2 = pd.DataFrame({"id": 2, 'date':'2022-05-26 08:02:47', 'employee': -1, 'action':-1}, index=[1262])
#train = pd.concat([train.iloc[:1261], first_row2, train.iloc[1261:]]).reset_index(drop=True)

train['date'] = train['date'].str.replace(':60',':59')
train['date'] = train['date'].str.replace(':61',':59')
train['date'] = train['date'].str.replace(':64',':59')
train['date'] = train['date'].str.replace(': ', ':')

train['date'] = pd.to_datetime(train['date']) #2022-05-24 13:59:60, 2022-05-27 13:55:60, 2022-05-27 13:55:61, 2022-05-27 13:55:61

In [6]:
startTime = train['date'].loc[0]
endTime = train['date'].loc[1261]
endTime

Timestamp('2022-05-26 08:02:49')

In [7]:
train['video_id'] = ''
train["video_id"].iloc[0:1261] = "train1.avi"
train["video_id"].iloc[1261:3521] = "train2.avi"

In [8]:
train1 = train[train['video_id'] == 'train1.avi']
position = train1.columns.get_loc('date')
train1['elapsed'] =  train1.iloc[1:, position] - train1.iat[0, position]
train1['seconds'] = train1.elapsed.dt.total_seconds()

train2 = train[train['video_id'] == 'train2.avi']
position = train2.columns.get_loc('date')
train2['elapsed'] =  train2.iloc[1:, position] - train2.iat[0, position]
train2['seconds'] = train2.elapsed.dt.total_seconds()

train = pd.concat([train1, train2], ignore_index=True)

In [9]:
#train = train.drop([0, 1261])
#train['seconds'] = train['seconds'].fillna(0)

In [10]:
train

,id,date,employee,action,video_id,elapsed,seconds
0,1256,2022-05-24 08:08:36,2,15.0,train1.avi,NaT,NaN
1,1257,2022-05-24 08:08:37,2,4.0,train1.avi,0 days 00:00:01,1.0
2,1258,2022-05-24 08:12:13,3,15.0,train1.avi,0 days 00:03:37,217.0
3,1259,2022-05-24 08:12:25,3,2.0,train1.avi,0 days 00:03:49,229.0
4,1260,2022-05-24 08:12:42,3,12.0,train1.avi,0 days 00:04:06,246.0
...,...,...,...,...,...,...,...
3514,4770,2022-05-28 07:42:51,0,4.0,train2.avi,1 days 23:40:02,171602.0
3515,4771,2022-05-28 07:44:27,0,14.0,train2.avi,1 days 23:41:38,171698.0
3516,4772,2022-05-28 07:44:36,0,3.0,train2.avi,1 days 23:41:47,171707.0
3517,4773,2022-05-28 07:46:21,0,13.0,train2.avi,1 days 23:43:32,171812.0


In [11]:
import os
train_dir = f"/content/gdrive/My Drive/video_OD/train/train_video"
image_dir = f"/content/gdrive/My Drive/video_OD/train/train_images/"
video_list = os.listdir(train_dir)
width = 1024
height = 1024
dim = (width, height)

In [ ]:
second = 0
for video in video_list:
    video_path = os.path.join(train_dir, video)
    vidcap = cv2.VideoCapture(video_path)
    while True:
      vidcap.set(cv2.CAP_PROP_POS_MSEC,second*1000)
      second += 1

      ret, frame = vidcap.read()
 
      frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)  

      if ret:
        cv2.imwrite(os.path.join(image_dir, f'second_{second}.jpg'), frame)

error: ignored

In [ ]:
def save_frame_sec(video_path, sec, result_path):
    """
    Save an image file from a video file.
    ----------------------------------------
    video_path(Path): Movie directory
    sec(float)       : Event detection time
    result_path(Path): Save directory
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return 

    fps = cap.get(cv2.CAP_PROP_FPS)

    cap.set(cv2.CAP_PROP_POS_MSEC, sec*1000)

    ret, frame = cap.read()

    if ret:
        cv2.imwrite(result_path, frame)

In [ ]:
def frames_by_ts():
  for video in video_list:
      video_path = os.path.join(train_dir, video)
      vidcap = cv2.VideoCapture(video_path)
      data_num = train.shape[0]
      for i in tqdm(range(data_num)):
          row = train.iloc[i]
          id = row.id
          time = row.seconds
          #employee = row.employee
          #action = row.action
          save_frame_sec(
              video_path, 
              second, 
              os.path.join(image_dir, f'second_{second}.jpg')
              )